In [3]:
import import_ipynb
from Pieces import Piece, GreenPiece, YellowPiece, BluePiece, MirrorPiece

In [3]:
from collections import namedtuple

Coord = namedtuple("Coord", ["X", "Y"]) 
Pos = namedtuple("Pos", ["X","Y"])

##have meta blocks like shrink, rotate, and ++ to augment powers of each block

In [4]:
import copy
class Board(object):
    """A class to hold the play space, location of placed blocks, and unused blocks"""
        
    
    def __init__(self, width=6, height=5, name="UNNAMED BOARD", owner="dev"):
        self.width = width; self.height = height
        self.name = name
        self.owner = owner
        self.activeLine = [Coord(i, 1) for i in range(self.width)] ###Hardcoded active line
        self.pieceIDs = {}
        self.reprDict = {0:"_", "ACTIVE":"#", "GREEN":"G" ,"BLUE":"B", "YELLOW":"Y"}
        
        
        
        #Build the main board and it's active line
        self.layout = [[0 for x in range(self.height)] for x in range(self.width)] 
        self.displayLayout = copy.deepcopy(self.layout) ###Maybe I don't really need a display board?
        self.SetLine(self.activeLine)
        
        #Initialize the board's first original state and the board that gets displayed.
        self.startingLayout = copy.deepcopy(self.layout)
        self.startingDisplay = copy.deepcopy(self.displayLayout)
        #self.UpdateDisplay()        
        
    def SetLine(self, coordList):
        """Currently a hardcoded active line. But at least it edits in place rather than creates a copy"""
        for coord in coordList:
            self.layout[coord.X][coord.Y] = 1
            self.displayLayout[coord.X][coord.Y] = "ACTIVE"
  
    def __repr__(self):
        return "<%s; w:%d h:%d>" %(self.name, self.width, self.height)
    
    def __str__(self):
        return "%s's %s" %(self.owner, self.name) 
    
    def PrintBoard(self):
        """Print a Command-Line, text base preview of the board layout."""
        for row in [*zip(*self.displayLayout)]: #transpose the display array since print doesnt work well with the xy order
            for block in row:    
                print("|%s|" %self.reprDict.get(block,"?"), end="")
            print("")
        print("")
            
    def UpdateDisplay(self): ###Not Used
        for coord in self.SetLine:
            self.displayLayout[coord.X][coord.Y] = "ACTIVE"
        for piece in self.pieceIDs.values():
            color = piece.GetColor()
            coordsList = piece.GetCoordinates()
            for coord in coordsList:
                self.displayLayout[coord.X][coord.Y] = color
                
    
    def ClearBoard(self):
        self.layout = copy.deepcopy(self.startingLayout)
        self.displayLayout = copy.deepcopy(self.startingDisplay)
        #self.SetLine()
        self.pieceIDs = {}
        
#--------------------------------------------------
    
    def GetPiece(self, placementCoord, displayValue=False):
        """Given a coordinate, return the ID of the piece at that location, or False if none exist."""
        if (not 0 <= placementCoord.X < self.width) or (not 0 <= placementCoord.Y < self.height):
            return False
        #if not displayValue: #we want the actual piece rather than its display values
        pieceID = self.layout[placementCoord.X][placementCoord.Y]
        if pieceID == 1 or pieceID == 0:
            return False
        return pieceID
        

    def PlacePiece(self, piece, placementCoord): #Should this just be combined with check placement?
        isLegal = self.CheckPlacement(piece, placementCoord)
        if isLegal:
            pieceID = piece.GetID()
            pieceColor = piece.GetColor()
            listCoordinates = isLegal
            if pieceID in self.pieceIDs.keys(): #Remove old placement if already on board
                self.RemovePiece(pieceID)
            #Update the piece's Coordinates with its new coordinates.
            piece.SetCoordinates(listCoordinates)

            self.pieceIDs[pieceID] = piece #place pieceID into board's dict of placed pieces
            for coord in listCoordinates:
                self.layout[coord.X][coord.Y] = pieceID
                self.displayLayout[coord.X][coord.Y] = pieceColor
            #for coord in piece:
            #    X = placementCoord.X + coord.X
            #    Y = placementCoord.Y + coord.Y
            #    self.layout[X][Y] = pieceID
            #    self.displayLayout[X][Y] = pieceColor
            return True
                
        return False
    ######maybe there should be a null piece check somewhere
    
        
    def CheckPlacement(self, piece, placementCoord, storeValues=True): #maybe later provide an option for not storing values in piece
        """Given a Piece w/Layout and a Coord for where to place the origin(0,0), check to see 
           that it doesn't overlap or go out of bounds.  If it is legal, return Bool True.  
           Additionaly if storeValues is true, return the new coordinates instead of True.
           """
        futureCoordinates = []
        for shapeCoord in piece:
            X = placementCoord.X + shapeCoord.X
            Y = placementCoord.Y + shapeCoord.Y
            if (not 0 <= X < self.width) or (not 0 <= Y < self.height):
                print("yeet1")########################
                return False
            if self.layout[X][Y] not in {0, 1, piece.GetID()}: #values rn are 1==empty and 2==activeLine
                print("yeet2: (%d, %d)" %(X, Y))
                return False
            futureCoordinates.append(Coord(X, Y))
        if storeValues:
            return futureCoordinates
        else:
            return True
        
    def HidePiece(self, piece):
        if isinstance(piece, Piece):
            pieceID = piece.GetID()
            ##Just remove the squares on the display board, but not the actual layout
            for coord in piece.GetCoordinates():
                self.displayLayout[coord.X][coord.Y] = self.startingDisplay[coord.X][coord.Y]
    
    def RemovePiece(self, piece):
        if isinstance(piece, Piece):
            pieceID = piece.GetID()
            ##remove the squares on board from piece.GetCoordinates()
            for coord in piece.GetCoordinates():
                self.layout[coord.X][coord.Y] = self.startingLayout[coord.X][coord.Y]
                self.displayLayout[coord.X][coord.Y] = self.startingDisplay[coord.X][coord.Y]
            del self.pieceIDs[pieceID]
            return piece
        elif isinstance(piece, int) and 9000 <= piece <= 9200:
            piece = self.pieceIDs[piece]
            return self.RemovePiece(piece)
        else:
            print("ohnoooooooooooooooTHROWERRORooooooooo")
            return False
        
        
    def Activate(self):
        value = 0
        activeIDList = [self.layout[coord.X][coord.Y] for coord in self.activeLine]
        #for coord in self.activeLine:
        #    pieceID = self.layout[coord.X][coord.Y]
        #    print(pieceID)
        #    activeIDList.append(pieceID)
        
        print(activeIDList)
        
        for i, ID in enumerate(activeIDList):
            if ID == 1:
                continue
            piece = self.pieceIDs[ID]
            value = piece.Activate(value, activeIDList[:i], activeIDList[i+1:], board=self)
            print("Score:", value)
        
        return value
        
        
                
            
        
          
            
b = Board()

NameError: name 'Coord' is not defined

In [6]:
Piece.ResetAllPieceIDs()
print(Piece.IDGenerator)

t = BluePiece([Coord(0,0), Coord(-1, 0), Coord(1, 0), Coord(0, 1)], pColor="BLUE")
u = GreenPiece([Coord(0,0), Coord(1, 0), Coord(0, 1), Coord(1, 1)])
v = GreenPiece([Coord(0,0), Coord(1, 0), Coord(0, 1), Coord(1, 1)])

t.Upgrade()
v.Upgrade()

t.SetOrientation(2)


print(u.shape)
print(t.GetID())
print(u.GetID())
print(v.GetID())
#for i in range(190):
#    t.__AssignID__()
print(Piece.IDGenerator)
type(Piece.IDGenerator)            



[9000, 9001, 9002, 9003, 9004, 9005, 9006, 9007, 9008, 9009]
[Coord(X=0, Y=0), Coord(X=1, Y=0), Coord(X=0, Y=-1), Coord(X=1, Y=-1)]
9009
9004
9008
[9000, 9001, 9002, 9003, 9005, 9006, 9007]


list

In [7]:
a = Board()
a.PrintBoard()
a.PlacePiece(t, Coord(3, 3))
print(a.pieceIDs.items())
a.PrintBoard()
a.ClearBoard()
a.PrintBoard()
a.PlacePiece(t, Coord(3, 3))
a.PrintBoard()
a.PlacePiece(t, Coord(1, 1))
a.PrintBoard()
a.PlacePiece(t, Coord(2, 1))
a.PrintBoard()
a.RemovePiece(t.GetID())
a.PrintBoard()
print(a.pieceIDs.items())

a.PlacePiece(t, Coord(2, 2))
u.SetOrientation(2)
a.PlacePiece(u, Coord(0, 0))
a.PrintBoard()
print(a.pieceIDs.items())

print(a.Activate())

v.SetOrientation(2)
a.PlacePiece(v, Coord(4, 0))
a.PrintBoard()
print(a.Activate())

a.PrintBoard()
v.Downgrade()
print(a.Activate())


|_||_||_||_||_||_|
|#||#||#||#||#||#|
|_||_||_||_||_||_|
|_||_||_||_||_||_|
|_||_||_||_||_||_|

dict_items([(9009, <Pieces.BluePiece object at 0x000002E26EE9EDC8>)])
|_||_||_||_||_||_|
|#||#||#||#||#||#|
|_||_||_||B||_||_|
|_||_||_||B||B||_|
|_||_||_||B||_||_|

|_||_||_||_||_||_|
|#||#||#||#||#||#|
|_||_||_||_||_||_|
|_||_||_||_||_||_|
|_||_||_||_||_||_|

|_||_||_||_||_||_|
|#||#||#||#||#||#|
|_||_||_||B||_||_|
|_||_||_||B||B||_|
|_||_||_||B||_||_|

|_||B||_||_||_||_|
|#||B||B||#||#||#|
|_||B||_||_||_||_|
|_||_||_||_||_||_|
|_||_||_||_||_||_|

|_||_||B||_||_||_|
|#||#||B||B||#||#|
|_||_||B||_||_||_|
|_||_||_||_||_||_|
|_||_||_||_||_||_|

|_||_||_||_||_||_|
|#||#||#||#||#||#|
|_||_||_||_||_||_|
|_||_||_||_||_||_|
|_||_||_||_||_||_|

dict_items([])
|G||G||_||_||_||_|
|G||G||B||#||#||#|
|_||_||B||B||_||_|
|_||_||B||_||_||_|
|_||_||_||_||_||_|

dict_items([(9009, <Pieces.BluePiece object at 0x000002E26EE9EDC8>), (9004, <Pieces.GreenPiece object at 0x000002E26EE9EA48>)])
[9004, 9004, 9009, 

In [8]:
a.ClearBoard()
a.PlacePiece(t, Coord(0, 1))
a.PlacePiece(u, Coord(2, 1))
v.Upgrade()
v.Downgrade()
a.PlacePiece(v, Coord(4, 1))
a.PrintBoard()
a.Activate()

|B||_||_||_||_||_|
|B||B||G||G||G||G|
|B||_||G||G||G||G|
|_||_||_||_||_||_|
|_||_||_||_||_||_|

[9009, 9009, 9004, 9004, 9008, 9008]
Score: 10
Score: 20
[9009, 9009] 9004 [9004, 9008, 9008]
Score: 40
[9009, 9009, 9004] 9004 [9008, 9008]
Score: 40
[9009, 9009, 9004, 9004] 9008 [9008]
Score: 80
[9009, 9009, 9004, 9004, 9008] 9008 []
Score: 80


80

In [9]:
a.ClearBoard()
t.SetOrientation(1)
for orientation in range(1,5):
    t.SetOrientation(orientation)
    print(t.shape)
    print(orientation)
    a.PlacePiece(t, Coord(3, 3))
    a.PrintBoard()

[Coord(X=0, Y=0), Coord(X=-1, Y=0), Coord(X=1, Y=0), Coord(X=0, Y=-1)]
1
|_||_||_||_||_||_|
|#||#||#||#||#||#|
|_||_||_||B||_||_|
|_||_||B||B||B||_|
|_||_||_||_||_||_|

[Coord(X=0, Y=0), Coord(X=0, Y=-1), Coord(X=0, Y=1), Coord(X=1, Y=0)]
2
|_||_||_||_||_||_|
|#||#||#||#||#||#|
|_||_||_||B||_||_|
|_||_||_||B||B||_|
|_||_||_||B||_||_|

[Coord(X=0, Y=0), Coord(X=1, Y=0), Coord(X=-1, Y=0), Coord(X=0, Y=1)]
3
|_||_||_||_||_||_|
|#||#||#||#||#||#|
|_||_||_||_||_||_|
|_||_||B||B||B||_|
|_||_||_||B||_||_|

[Coord(X=0, Y=0), Coord(X=0, Y=1), Coord(X=0, Y=-1), Coord(X=-1, Y=0)]
4
|_||_||_||_||_||_|
|#||#||#||#||#||#|
|_||_||_||B||_||_|
|_||_||B||B||_||_|
|_||_||_||B||_||_|



In [10]:
class QI:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return "QI " + self.name
    def clear(self):
        self.name = "Accropolis"
    def addition(self, value):
        return value + 10
    def echo(self, value):
        return value

    
alan = QI("alan") #Create instance of Alan
fry = QI("fry")

cast = {} #Dictionary of class objects
cast[0] = alan #set dict 0:alan
cast[1] = fry

alan.clear() #Change alan's name to Accropolis
alan 
cast #cast also changed, so we sent a refference of alan and not a copy

{0: QI Accropolis, 1: QI fry}

In [11]:
class Fry(QI):
    def __repr__(self):
        return "Host " + self.name
    
    def addition(self, value):
        return value+100
    
alan = QI("alan")
fry = Fry("fry")
fry
alan
fry.addition(50)
alan.addition(50)
alan.echo("greetings")
fry.echo("greetings")


'greetings'

In [12]:
t1 = Piece([Coord(0,0), Coord(-1, 0), Coord(1, 0), Coord(0, 1)])
u1 = Piece([Coord(0,0), Coord(1, 0), Coord(0, 1), Coord(1, 1)])
print(t1.shape)

[Coord(X=0, Y=0), Coord(X=-1, Y=0), Coord(X=1, Y=0), Coord(X=0, Y=-1)]


In [13]:
b = Board()
b.PrintBoard()
t.SetOrientation(3)
b.PlacePiece(t, Coord(1,1))
b.PrintBoard()
t.Upgrade()
print(t.upgraded)
b.Activate()

|_||_||_||_||_||_|
|#||#||#||#||#||#|
|_||_||_||_||_||_|
|_||_||_||_||_||_|
|_||_||_||_||_||_|

|_||_||_||_||_||_|
|B||B||B||#||#||#|
|_||B||_||_||_||_|
|_||_||_||_||_||_|
|_||_||_||_||_||_|

True
[9009, 9009, 9009, 1, 1, 1]
Score: 10
Score: 20
Score: 30


30